# 크롤링으로 위경도 가져오기

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import random
import time
from tqdm import tqdm, trange

rent_stations = []
base_url = 'https://www.bikeseoul.com/app/station/moveStationSearchView.do?currentPageNo='

def crawling_rent_station(start, end):
    for num in trange(start, end+1):
        page_source = requests.get( base_url + str(num))
        soup = bs(page_source.text, 'html.parser')
        loc_table = soup.select('table.psboard1 > tbody')[0].find_all('tr')
        for row in loc_table:
            loc_info = []
            loc_name = row.select('td.pl10')[0].get_text(strip=True)
            if '.' in loc_name:
                loc_info.append(loc_name.split('.')[-1].lstrip())
            else:
                loc_info.append(loc_name)
            # 운영여부
            loc_info.append(row.select('td.pl10')[1].get_text(strip=True))
            # 주소
            loc_info.append(row.select('td.mhid')[0].get_text(strip=True))
            # 위도, 경도
            loc_geo = row.find('a')['param-data'].split(',')
            loc_info.append(loc_geo[0])
            loc_info.append(loc_geo[1])
            # 리스트에 대여소 추가
            rent_stations.append(loc_info) 
        time.sleep(random.randint(1,3))
    return rent_stations

In [2]:
# 처음엔 한 두 페이지정도를 크롤링해보고 정상으로 동작하면 전체를 크롤링한다.
# rent_stations = crawling_rent_station(1, 2)
rent_stations = crawling_rent_station(1, 293)

100%|██████████| 293/293 [13:17<00:00,  2.72s/it]


In [3]:
import pandas as pd
header = ['대여소','상태','주소','위도','경도']
df = pd.DataFrame.from_records(rent_stations, columns = header)
df.shape

(1461, 5)

In [4]:
df.head()

,대여소,상태,주소,위도,경도
0,광진교 남단 사거리(디지털프라자앞),운영중,서울특별시 강동구 구천면로 171 376 남단사거리,37.541805,127.124718
1,해공공원(천호동),운영중,서울특별시 강동구 올림픽로 702 265,37.545219,127.125916
2,해공도서관앞,운영중,서울특별시 강동구 올림픽로 702 367,37.543915,127.125458
3,삼성광나루아파트 버스정류장,운영중,서울특별시 강동구 상암로3길 77 암사동 534,37.553349,127.128860
4,롯데캐슬 115동앞,운영중,서울특별시 강동구 양재대로 1665 명일동 304,37.554920,127.142799


In [5]:
df.to_csv('data/bike_rent_station.csv', index=False)

In [6]:
# 파일이 제대로 생성되었는지 확인
%ls data

bike_rent_station.csv
bike_rent_station_gu.csv
seoul-bike-2018.csv
서울_공공자전거_따릉이_이용현황자료_20161026.xlsx
서울특별시 공공자전거 대여이력 정보_2015년.csv
서울특별시 공공자전거 대여이력 정보_2016년_1.csv
서울특별시 공공자전거 대여이력 정보_2016년_2.csv
서울특별시 공공자전거 대여이력 정보_2016년_3.csv
서울특별시 공공자전거 대여이력 정보_2017년_1.csv
서울특별시 공공자전거 대여이력 정보_2017년_2분기_1.csv
서울특별시 공공자전거 대여이력 정보_2017년_2분기_2.csv
서울특별시 공공자전거 대여이력 정보_2017년_3분기_1.csv
서울특별시 공공자전거 대여이력 정보_2017년_3분기_2.csv
서울특별시 공공자전거 대여이력 정보_2017년_4분기_1.csv
서울특별시 공공자전거 대여이력 정보_2017년_4분기_2.csv
서울특별시 공공자전거 대여이력 정보_2018년_1분기1.csv
서울특별시 공공자전거 대여이력 정보_2018년_2분기_1.csv
서울특별시 공공자전거 대여이력 정보_2018년_2분기_2.csv
서울특별시 공공자전거 대여이력 정보_2018년_2분기_3.csv
서울특별시 공공자전거 대여이력 정보_2018년_2분기_4.csv
